In [3]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import yfinance as yf
from helpers import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### SPX OHLC

In [ ]:
spx = yf.Ticker('^SPX')
spx_history = spx.history(period='2y')
spx_history

#### ATM Options

In [ ]:
date_range = pd.date_range(start="2023-01-03", end = "2024-06-30", freq='D')
df_concat = pd.DataFrame()

for date in date_range:
    date_hyphens = date.strftime('%Y-%m-%d')
    date_no_hyphens = date.strftime('%Y%m%d')

    if date_hyphens not in spx_history.index:
        print(date_hyphens + " OHLC not found")
        continue
    else:
        print(f"{date_hyphens}")
        row = spx_history.loc[date_hyphens]
        open = row['Open']
        print(f"\tOpen:  {open:.2f}")

        strikes = pd.read_csv(f"/data/thetadata/options/spxw/strikes/{date_no_hyphens}.csv.gz", compression='gzip')
        atm_strike = strikes.loc[(strikes.sub(Decimal(str(open * 1000))).abs().idxmin())]
        atm_strike = atm_strike.iloc[0].strike
        print(f"\tATM Strike: {atm_strike}")

        interval = '1m'
        df = pd.read_csv(f"/data/thetadata/options/spxw/0dte/{interval}/{date_no_hyphens}.csv.gz", compression='gzip')
        df = df[df['strike'] == atm_strike]
        expected_rows = 782
        if (len(df) != expected_rows):
            print(f"{date_no_hyphens} expected {expected_rows} rows but got {df.rows.count()}")
        print(f"\tOption prices: {len(df)}")

        df_concat = pd.concat([df_concat, df])

df = df_concat.copy()

### Prepare data

In [118]:
# Add columns
df['mid'] = round((df['bid'] + df['ask']) / 2, 4)
# df['vbid'] = round((df['bid'] * df['bid_size']) /2, 4)
# df['vask'] =  round((df['ask'] * df['ask_size']) /2, 4)

# Drop columns
df.drop(columns=['expiration', 'root', 'bid_exchange', 'bid_condition', 'ask_exchange', 'ask_condition'], inplace=True)
df.drop(columns=['bid', 'bid_size', 'ask', 'ask_size'], inplace=True)
df.drop(columns=['Unnamed: 0'], inplace=True)
df.drop_duplicates(inplace=True)

# Pivot bid/ask from separate rows to columns
pivot_df = df.pivot_table(index=['date', 'ms_of_day'], columns='right', values='mid', aggfunc='first')
pivot_df.columns = ['call_mid', 'put_mid'] # Rename the columns
pivot_df = pivot_df.reset_index() # Reset the index

# Remove opening interval and set datetime index
pivot_df = pivot_df[pivot_df['ms_of_day'] != 34200000]
pivot_df['ts'] = pd.to_datetime(pivot_df['date'], format='%Y%m%d') + pd.to_timedelta(pivot_df['ms_of_day'], unit='ms')
pivot_df.set_index('ts', inplace=True)

print(f"Number of quotes loaded: {len(pivot_df)}")
pivot_df

Number of quotes loaded: 145860


,date,ms_of_day,call_mid,put_mid
ts,,,,
2023-01-03 09:31:00,20230103,34260000,16.900,16.10
2023-01-03 09:32:00,20230103,34320000,15.850,16.50
2023-01-03 09:33:00,20230103,34380000,16.600,15.05
2023-01-03 09:34:00,20230103,34440000,21.100,10.85
2023-01-03 09:35:00,20230103,34500000,17.700,12.85
...,...,...,...,...
2024-06-28 15:56:00,20240628,57360000,0.025,24.10
2024-06-28 15:57:00,20240628,57420000,0.025,27.20
2024-06-28 15:58:00,20240628,57480000,0.025,24.90
